# CSCI 3352 Biological Networks, Spring 2022, Prof. Clauset

# In-class lab : metabolic flux balance analysis

***

**Team names**: 

***

This is an in-class "laboratory," in which you will learn about flux balance analysis, visualize the results of optimizing a FBA model for biomass growth, and perform and visualization the impact of a "knock out" experiment.

**To begin**: follow the [installation instructions](https://github.com/opencobra/cobrapy/blob/master/INSTALL.rst) for the `CobraPy` package.

![alt text](https://aaronclauset.github.io/courses/3352/cobrapy_header.png "CobraPy header")

In [1]:
import pandas
import cobra
import cobra.test

## Part 1. Going with the flux

This first section of the lab follows steps outlined in Keesha Erickson's "[FBA with CobraPy](https://cnls.lanl.gov/external/qbio2018/Slides/FBA%202/qBio-FBA-lab-slides.pdf)" tutorial from June 2018. In this section, you will
* load an E. coli metabolic network model
* grab the list of reactions it contains
* set up and visualization the results of a basic FBA experiment

NB: The middle part of the tutorial, on overproducing lycopene, is beyond the scope of the lab, and a few of the variable names in this tutorial did not line up with the package when I ran it.


In [ ]:
# OPTIONAL : If you'd like, run the recommended CobraPy test function to see all the tests it runs
#from cobra.test import test_all
#result = test_all()
#print(result)

### 1a. Load a test model
CobraPy comes with several "test" models built in. We'll work with the *E. coli* model iJO1366 at first.

In [2]:
# Load the model for genome scale E. coli iJO1366
# 1367 genes, 2583 reactions, and 1805 metabolites
#
# BioModel page: https://www.ebi.ac.uk/compneur-srv/biomodels-main/MODEL1108160000
# From: Orth et al. "A comprehensive genome-scale reconstruction of Escherichia coli metabolism-2011." Mol. Syst. Biol. 7, 535 (2011).
# DOI: 10.1038/msb.2011.65

model = cobra.test.create_test_model("ecoli")

# Let's see what's in this model
print(f'%i reactions' % len(model.reactions))
print(f'%i metabolites' % len(model.metabolites))
print(f'%i genes' % len(model.genes))

# NB: There are 2606 whole genome metabolism models available from EMBL:
# https://www.ebi.ac.uk/biomodels-main/path2models?cat=genome-scale

2583 reactions
1805 metabolites
1367 genes


Inside the CobraPy `model` data structure many substructures. Some of the ones we will use include:
* `.reactions`: A DictList where the key is the reaction identifier and the value a Reaction
* `.metabolites`: A DictList where the key is the metabolite identifier and the value a Metabolite
* `.genes`: A DictList where the key is the gene identifier and the value a Gene
* `.solution`: The last obtained solution from optimizing the model.
* `.objective`: Get or set the solver objective.


Let's look at the set of reactions included in this model.
(This will be useful because some of the names of these reactions differ from the tutorial. Can you find the right substitutions?)

In [3]:
for rxn in model.reactions:
    print(rxn)
# Given these reactions, you could transform this data into a directed, bipartite metabolite-reaction network

DM_4crsol_c: 4crsol_c --> 
DM_5drib_c: 5drib_c --> 
DM_aacald_c: aacald_c --> 
DM_amob_c: amob_c --> 
DM_mththf_c: mththf_c --> 
DM_oxam_c: oxam_c --> 
BIOMASS_Ec_iJO1366_WT_53p95M: 0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c + 0.234232 asn__L_c + 0.234232 asp__L_c + 54.119975 atp_c + 0.000116 bmocogdp_c + 2e-06 btn_c + 0.004952 ca2_c + 0.000223 chor_c + 0.004952 cl_c + 0.002944 clpn160_p + 0.00229 clpn161_p + 0.00118 clpn181_p + 0.000168 coa_c + 2.4e-05 cobalt2_c + 0.008151 colipa_e + 0.129799 ctp_c + 0.000674 cu2_c + 0.088988 cys__L_c + 0.024805 datp_c + 0.025612 dctp_c + 0.025612 dgtp_c + 0.024805 dttp_c + 0.000223 enter_c + 0.000223 fad_c + 0.006388 fe2_c + 0.007428 fe3_c + 0.255712 gln__L_c + 0.255712 glu__L_c + 0.595297 gly_c + 0.154187 glycogen_c + 0.000223 gthrd_c + 0.209121 gtp_c + 48.752916 h2o_c + 0.000223 hemeO_c + 0.092056 his__

MN6PP: h2o_c + man6p_c --> man_c + pi_c
MNLptspp: mnl_p + pep_c --> mnl1p_c + pyr_c
MNLtex: mnl_e <=> mnl_p
MNNH: mana_c --> 2ddglcn_c + h2o_c
MNt2pp: h_p + mn2_p --> h_c + mn2_c
MNtex: mn2_e <=> mn2_p
MOADSUx: iscssh_c + moadamp_c + nadh_c --> amp_c + iscs_c + moadcosh_c + nad_c
MOAT: ckdo_c + lipidA_c --> cmp_c + h_c + kdolipid4_c
MOAT2: ckdo_c + kdolipid4_c --> cmp_c + h_c + kdo2lipid4_c
MOAT3C: ckdo_c + phphhlipa_c --> cmp_c + h_c + kphphhlipa_c
MOBDabcpp: atp_c + h2o_c + mobd_p --> adp_c + h_c + mobd_c + pi_c
MOBDtex: mobd_e <=> mobd_p
MOCDS: ctp_c + h_c + moco_c --> mococdp_c + ppi_c
MOCOS: 2.0 h_c + mobd_c + mptamp_c --> amp_c + cu2_c + h2o_c + moco_c
MOGDS: gtp_c + h_c + moco_c --> mocogdp_c + ppi_c
MOHMT: 3mob_c + h2o_c + mlthf_c --> 2dhp_c + thf_c
MOX: mal__L_c + o2_c <=> h2o2_c + oaa_c
MPTAT: atp_c + h_c + mpt_c --> mptamp_c + ppi_c
MPTG: 2.0 uaagmda_c --> 2.0 h_c + murein5p5p_p + 2.0 udcpdp_c
MPTG2: murein5p5p_p + uaagmda_c --> h_c + murein5p5p5p_p + udcpdp_c
MPTS: cpmp_c +

At the top of the `model.reactions` dict are two BIOMASS reactions: `BIOMASS_Ec_iJO1366_WT_53p95M` and `BIOMASS_Ec_iJO1366_core_53p95M`. If you look carefully at their definition, you can see both the large number of input metabolites that BIOMASS production requires, and the very precise coefficients attached to each. These values are experimentally derived to reflect how the model's organism grows in practice, so that model predictions have the right scale for being tested in the laboratory.

When we "solve" the model, we are asking the optimizer to choose a set of `.fluxes` on the reactions that maximize some `objective`. The objective is usually flux through some particular reaction. The BIOMASS reactions are a common choice for these, but we could choose any combination of reactions in the model. Let's look at what direction the solver will optimize, and what objective function it's optimizing:

In [4]:
print(model.objective)

Maximize
1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1


### 1b. Optimize the model, get some fluxes
The basic experiment we run is to optimize the `.fluxes`, i.e., choose the set of flow values on all the network's reactions so that the `objective` is optimized (maximized, in this case).

To do this, we
* set the *objective*
* set *minimum* fluxes on our input nutrients
* `.optimize()` the model

We can the output the `.objective_value` the solver produces along with a `.summary()` of the model.

In [14]:
# Set the objective to the genome scale biomass reactions
# NB: that these reaction names are different than in the tutorial
model.reactions.get_by_id("BIOMASS_Ec_iJO1366_core_53p95M").objective_coefficient = 0
model.reactions.get_by_id("BIOMASS_Ec_iJO1366_WT_53p95M").objective_coefficient = 1.0

# Set constraints for aerobic growth in glucose minimal media
# NB: that these reaction names are different than in the tutorial
model.reactions.get_by_id("EX_glc__D_e").lower_bound = -10
model.reactions.get_by_id("EX_o2_e").lower_bound = -15

# Solve
solution = model.optimize()

# Output solution
print(f'Growth Rate: '+str(solution.objective_value)+' 1/h')

# Output more information
model.summary()

Growth Rate: 0.9009127876082755 1/h


Before proceeding, discuss the following questions with your partner:
* How close is the growth rate to what is given in the tutorial and lecture?
* Is the ecoli metabolism using all the inputs you gave it?
* What metabolic waste products is it producing?
* Is this metabolism aerobic or anaerobic?

### 1c. Visualize the fluxes

To see what `.fluxes` the model actually chose for the different edges in *E. coli*'s metabolism, we'll need to use the `Escher` tool. Esher is another python package, but it doesn't seem to work well inside Jupyter notebooks, so instead we'll use the website version:

1. Write out the `solution.fluxes` data as a csv file
* Visit the [Escher website](https://escher.github.io/#/) and load the E. coli map, or just [load the map directly](https://escher.github.io/#/app?map=e_coli_core.Core%20metabolism&tool=Builder&model=e_coli_core) in a new browser window.
* Use the `Data` menu to `Load reaction data`
* Choose the `FBA_max_biomass.csv` file
* Voila. A beautiful visualization of the fluxes your solver produced.

You can also load the "central metabolism iJ01366" map, which shows a larger portion of the metabolic network in the model we've loaded.

In [12]:
# Write out the solution.fluxes data as a csv so that we can import it into an Escher visualization
# NB: the fluxes are in solution.fluxes, unlike in the tutorial
df = pandas.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('FBA_max_biomass.csv')

Now that we've put the `solution.fluxes` information into a `pandas` dataframe, we can also just write it out to see which reactions have some flux across them. From this, you can see that most reactions have very little flux across them (lots of 0s). The *absolute values* of these fluxes is not all that meaningful, since they are just the numerical solutions from the solver. Only the flux across the BIOMASS reactions are scaled appropriately to be biologically meaningful.

In [13]:
df

,fluxes
DM_4crsol_c,0.000201
DM_5drib_c,0.000208
DM_aacald_c,0.000000
DM_amob_c,0.000002
DM_mththf_c,0.001207
...,...
ZN2abcpp,0.000000
ZN2t3pp,0.000000
ZN2tpp,0.000292
ZNabcpp,0.000000


Once you've loaded the flux `.csv` into Escher, you can see where the flows are going. Since Escher doesn't like to run in the notebook, here's a screenshot of what it produces:

![alt text](https://aaronclauset.github.io/courses/3352/Escher_ecoli_small.png "Ecoli small")

### 1d. Explore basic metabolism
Using the basic ecoli metabolism model above, explore the following questions. For each, first discuss what kind of experiment you should carry out in order to answer it, and then try it out. Record what you find.
* What is the optimum ratio of o2 availability per unit of glucose consumption for maximizing biomass production?
* How much more effective is aerobic metabolism (o2 input > 0) vs. anaerobic metabolism (no o2) for biomass production?

## Part 2. Knock out experiments
In this section, we'll carry out a "knockout" experiment, in which we force some particular flux in the system to be zero. Experimentally, we would accomplish this by deleting or disabling the gene that codes for the enzyme that catalyzes that reaction. Here, we can just add a constraint to the model that prohibits all solutions that place any flux through a particular reaction. This will force the flows to go across other reactions to get to BIOMASS, which may decrease (or increase) BIOMASS production.

In `CobraPy` there are two ways to do a knockout:
* `model.reactions.PFK.knock_out()`
* `model.genes.b4025.knock_out()`

The *effect* of a `reactions` knockout is to set *both* `model.reactions.get_by_id("x").lower_bound = 0` *and* `model.reactions.get_by_id("x").upper_bound = 0`, which prevents flux from going in either direction across the reaction.

Once we've done the knockout, we need to re-solve the model to get *new* fluxes, which respect the added constraint of having eliminated this reaction. Then we can re-export the `.fluxes` and visualize them with Escher again to see how the metabolic flows shifted.

We'll use the same `.obective` function as before, along with the same aerobic growth in glocuse conditions.

### Part 2a. Knocking out FUM

This part is mostly an example of how to set up and run a knockout experiment. Run the code and then discuss these questions with your group:
* How much lower is the growth rate in this condition?
* What changes in the Escher flux visualization do you observe? (What fluxes increased? What fluxes decreased?)
* Has this knockout experiment changed the ratio of o2 availability per unit glucose consumption?

In [16]:
# Carry out a knock-out experiment:
# https://cobrapy.readthedocs.io/en/stable/deletions.html

# knock out the FUM reaction, which is part of the Krebs cycle
model.reactions.FUM.knock_out()

# Set the objective to the genome scale biomass reactions
# NB: that these reaction names are different than in the tutorial
model.reactions.get_by_id("BIOMASS_Ec_iJO1366_core_53p95M").objective_coefficient = 0.0
model.reactions.get_by_id("BIOMASS_Ec_iJO1366_WT_53p95M").objective_coefficient = 1.0

# Set constrants for aerobic growth in glucose minimal media
# NB: that these reaction names are different than in the tutorial
model.reactions.get_by_id("EX_glc__D_e").lower_bound = -10
model.reactions.get_by_id("EX_o2_e").lower_bound = -15

# Solve
solution = model.optimize()

# Output solution
print(f'Growth Rate: '+str(solution.objective_value)+' 1/h')

# Write out the solution.fluxes data as a csv so that we can import it into an Escher visualization
df = pandas.DataFrame.from_dict([solution.fluxes]).T
df.to_csv('FBA_max_biomass_noFUM.csv')

# Output more information
model.summary()

Growth Rate: 0.8477618226313935 1/h


Using the Escher tool again, below is another screenshot of the resulting flows in core metabolism:

![alt text](https://aaronclauset.github.io/courses/3352/Escher_ecoli_noFUM_small.png "Ecoli small, no FUM")

### Part 2b. Explore!

Now, try knocking out other reactions and answering these questions:
* Can you find any "bottleneck" reactions, which, if you knock them out, drop growth rate to nearly 0?
  * Where would you _expect_ bottleneck reactions to be in the network?
  * Do all those reactions have the same impact if knocked out?
  * How many bottlenecks can you find?
  * What does the common-ness or rarity of bottlenecks tell you about the _robustness_ of the metabolic network?
* Can you find any reactions that, if you knock them out, lead to _increased_ growth rate?
* Can you find reactions that, if knocked out, increase the o2 consumption per unit of glucose?

### 2c. More ideas

There are many other experiments you could try. The growth conditions we've set so far include minimal glucose. Increasing the glucose supply should increase the flux through to BIOMASS. How high a growth rate can you sustain? How many reactions can you knock out and still get some growth? How much do "core" vs. "periphery" reactions matter for altering the growth rate?

You can also try optimizing flux through different reactions by altering the `.objective` function. What happens to growth if you optimize for flux around the Krebs cycle?

***
## Other metabolic models

As far as I know, there are not pre-built Escher maps for these metabolisms.

In [ ]:
# Here are two other models included with CobraPy
textbook_model = cobra.test.create_test_model("textbook")
# Let's see what's in this model:
print(f'textbook model')
print(f'%i reactions' % len(textbook_model.reactions))
print(f'%i metabolites' % len(textbook_model.metabolites))
print(f'%i genes\n' % len(textbook_model.genes))

salmonella_model = cobra.test.create_test_model("salmonella")
# Let's see what's in this model:
print(f'salmonella model')
print(f'%i reactions' % len(salmonella_model.reactions))
print(f'%i metabolites' % len(salmonella_model.metabolites))
print(f'%i genes' % len(salmonella_model.genes))